In [30]:
import requests
env = open('../.env').readlines()
DISCORD_APPLICATION_ID = [x.split('=')[1].strip().replace('"', '') for x in env if x.startswith('DEV_DISCORD_APPLICATION_ID')][0]
DISCORD_BOT_TOKEN = [x.split('=')[1].strip().replace('"', '') for x in env if x.startswith('DEV_DISCORD_BOT_TOKEN')][0]
headers = {
    "Authorization": f"Bot {DISCORD_BOT_TOKEN}"
}

# LIST COMMANDS

In [13]:
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"
r = requests.get(url, headers=headers)
r.json()

[{'id': '920656302420684820',
  'application_id': '920612693864493066',
  'version': '937447797664403537',
  'default_permission': True,
  'default_member_permissions': None,
  'type': 1,
  'name': 'starter',
  'description': 'Generate a conversation starter given some topics',
  'dm_permission': None,
  'options': [{'type': 3,
    'name': 'topics',
    'description': "List of topics separated by ','. Example 'ice breaker,travel,dating'. (default: 'ice breaker')"},
   {'type': 3,
    'name': 'players',
    'description': "List of challenged players separated by ','. Example '@user1,@user2'"}]},
 {'id': '937361355932070030',
  'application_id': '920612693864493066',
  'version': '937361355932070031',
  'default_permission': True,
  'default_member_permissions': None,
  'type': 1,
  'name': 'about',
  'description': 'Give information about the bot',
  'dm_permission': None},
 {'id': '937372525892096011',
  'application_id': '920612693864493066',
  'version': '937402177402601472',
  'defa

# /starter

In [12]:
d = {
    "name": "starter",
    "description": "Generate a conversation starter given some topics",
    "options": [
        {
            "name": "topics",
            "description": "List of topics separated by ','."
            + " Example 'ice breaker,travel,dating'. (default: 'ice breaker')",
            "type": 3,
            "required": False,
        },
        {
            "name": "players",
            "description": "List of challenged players separated by ','. Example '@user1,@user2'",
            "type": 3,
            "required": False,
        }
    ]
}
COMMAND_ID="920656302420684820"
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands/"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '920656302420684820',
 'application_id': '920612693864493066',
 'version': '937447797664403537',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'starter',
 'name_localizations': None,
 'description': 'Generate a conversation starter given some topics',
 'description_localizations': None,
 'dm_permission': None,
 'options': [{'type': 3,
   'name': 'topics',
   'name_localizations': None,
   'description': "List of topics separated by ','. Example 'ice breaker,travel,dating'. (default: 'ice breaker')",
   'description_localizations': None},
  {'type': 3,
   'name': 'players',
   'name_localizations': None,
   'description': "List of challenged players separated by ','. Example '@user1,@user2'",
   'description_localizations': None}]}

# COMMAND DELETE

In [13]:
COMMAND_ID = ""
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands/{COMMAND_ID}"
r = requests.delete(url, headers=headers)
r.text

''

# /about

Langame
A game where you can play with your friends.
How to play
Send this command to a channel:
/starter topics:ice breaker,travel,whatever topic your like

You can also add players to the Langame by adding the following option:
/starter players:@user1,@user2,@user3

With love, by https://langa.me/. Augmented conversations.


In [7]:

d = {
    "name": "about",
    "description": "Give information about the bot",
}
COMMAND_ID = ""
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands/"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

'{"id": "937361355932070030", "application_id": "920612693864493066", "version": "937361355932070031", "default_permission": true, "default_member_permissions": null, "type": 1, "name": "about", "name_localizations": null, "description": "Give information about the bot", "description_localizations": null, "dm_permission": null}'

# /sub

/sub topics:ice breaker,travel players:@louis030195,@bob27 frequency:12h

results in the bot posting conversation starters between these two every 12h on the given topics

Optionally use players:random to have the bot choose a random player from the server


### Execution

Topics:ice breaker,travel
Players:@louis030195,@bob27

What is your favorite place?

CTA component button to stop the sub, like button click on /unsub whatever

In [41]:
d = {
    "name": "sub",
    "description": "Subscribe to frequent Langames",
    "options": [
        {
            "name": "topics",
            "description": "List of topics separated by ','. Example 'ice breaker,travel,dating'"
            + "(default 'ice breaker')",
            "type": 3,
            "required": False,
        },
        # {
        #     "name": "players",
        #     "description": "List of challenged players separated by ','."
        #     + " Example '@user1,@user2'. (default is random players)",
        #     "type": 3,
        #     "required": False,
        # },
        {
            "name": "frequency",
            "description": "Frequency in minutes,"
            + " hours or days. Example '30m' or '5h' or '2d'"
            + " (default is 3h)",
            "type": 3,
            "required": False,
        }
    ],
}
COMMAND_ID = "937372525892096011"
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands/"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '937372525892096011',
 'application_id': '920612693864493066',
 'version': '937459048939417600',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'sub',
 'name_localizations': None,
 'description': 'Subscribe to frequent Langames',
 'description_localizations': None,
 'dm_permission': None,
 'options': [{'type': 3,
   'name': 'topics',
   'name_localizations': None,
   'description': "List of topics separated by ','. Example 'ice breaker,travel,dating'(default 'ice breaker')",
   'description_localizations': None},
  {'type': 3,
   'name': 'frequency',
   'name_localizations': None,
   'description': "Frequency in minutes, hours or days. Example '30m' or '5h' or '2d' (default is 3h)",
   'description_localizations': None}]}

# /unsub

Unsubscribe the server from Langames.

In [7]:
d = {
    "name": "unsub",
    "description": "Unsubscribe from frequent Langames",
}
COMMAND_ID = None
url = f"https://discord.com/api/v8/applications/{DISCORD_APPLICATION_ID}/commands"+(f"/{COMMAND_ID}" if COMMAND_ID else "")
r =  requests.patch(url, headers=headers, json=d) if COMMAND_ID else requests.post(url, headers=headers, json=d)
r.json()

{'id': '937402122624962611',
 'application_id': '920612693864493066',
 'version': '937402122624962612',
 'default_permission': True,
 'default_member_permissions': None,
 'type': 1,
 'name': 'unsub',
 'name_localizations': None,
 'description': 'Unsubscribe from frequent Langames',
 'description_localizations': None,
 'dm_permission': None}